<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


## Author: Lyndsey Pohl

# Assignment 11

This assignment is probably the most involved yet.  You will process a large file through Python, and then pass it to R for statistical analysis.

We will analyze Amazon reviews to determine what characteristics make them most helpful.

## Problem 1(a).  Reading Amazon Reviews.

🎯 Download the file of Amazon gourmet food reviews from the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/web-FineFoods.html).   (Your computer may already have a utility installed that can unzip the archive as a text file; if not, [7-zip](http://www.7-zip.org/) is a useful utility for Windows. You can also use an online utility by doing a web search for: ``open .gz files online``.)

---

### Do not skip the following step--if you skip it, you will create headaches for yourself when trying to submit your assignment.
The data file is too large to upload to GitHub, so before we commit any changes to our code, we need to add the data file to the `.gitignore` file.  This way, git will ignore the data file and not try to upload it.

🎯 Open the `.gitignore` file in your favorite text editor.  Add the following lines:  
`finefoods.txt.gz`  
`finefoods.csv`  
`foods.txt`  
and any other filenames of very large data files inside the folder where you downloaded your fork of the GitHub repository.  Save the `.gitignore` file.
### Do not skip the step described above!
---

🎯 Create a pandas DataFrame object with the following entries for each review:

1. Product ID
2. Number of people who voted this review helpful
3. Total number of people who rated this review
4. Reviewer's score rating of the product
5. Text of the review -- this will be dropped before you write your data file and port to R.

For the second and third of these, the information will be given in the file as ```1/5```, which would correspond to 1 vote for helpful out of 5 people who rated the review.

---

###### Examples

Consider this review:

    product/productId: B001E4KFG0
    review/userId: A3SGXH7AUHU8GW
    review/profileName: delmartian
    review/helpfulness: 1/1
    review/score: 5.0
    review/time: 1303862400
    review/summary: Good Quality Dog Food
    review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

The fraction for `review/helpfulness` for delmartian's review here is 1/1.  One person voted on whether it was a helpful review, and that one person found it helpful.

---

    product/productId: B001GVISJM
    review/userId: A2MUGFV2TDQ47K
    review/profileName: Lynrie "Oh HELL no"
    review/helpfulness: 4/5
    review/score: 5.0
    review/time: 1268352000
    review/summary: Strawberry Twizzlers - Yummy
    review/text: The Strawberry Twizzlers are my guilty pleasure - yummy. Six pounds will be around for a while with my son and I.

This review was rated for helpfulness by a total of 5 people, but only four of them found it helpful -- the other person didn't think it was a useful review.

---

    product/productId: B001GVISJM
    review/userId: A3IV7CL2C13K2U
    review/profileName: Greg
    review/helpfulness: 0/0
    review/score: 5.0
    review/time: 1318032000
    review/summary: Home delivered twizlers
    review/text: Candy was delivered very fast and was purchased at a reasonable price.  I was home bound and unable to get to a store so this was perfect for me.
    
Zero people voted on whether Greg's review was helpful.  Of these zero people, zero found it helpful. 

---

###### Notes

I strongly suggest that you take some smaller portion of the file off the top for testing and development.  On a Mac, in the [terminal](https://www.macworld.co.uk/feature/mac-software/how-use-terminal-on-mac-3608274/), change directories to the location of this repo, and run

    head -n 500 finefoods.csv > finefoods_test.csv
    
But, change that 500 to some number such that you don't have a broken review at the bottom!  Windows students, use the [Powershell](https://en.wikipedia.org/wiki/PowerShell). (regular old `cmd` is terrible)

---

Here are the first 10 reviews, correctly parsed:

![first_10](first_10.jpg)

In [1]:
%%time
import pandas as pd
cols = ['Product ID', '# People Helpful', 'Total Reviews', 'Score','Review']
data_df = pd.DataFrame(columns=cols)

file = 'foods.txt'
data_foodtext=[]
with open(file, 'r', encoding = 'latin-1') as f:  
    for line in f:
        if line != '\n':
            data_foodtext.append(line)





data_foodtext = [item for line in data_foodtext for item in line.split(':')]


for index, item in enumerate(data_foodtext):
    data_foodtext[index] = str(item).replace('\n','')

    


#some reviews contain ':' so need to search for those split reviews and combine them together again
#I don't think the following part of the code is as efficiecnt as it should be, but I cannot think of another way to split the data by not using ':'
#This part of the code adds considerable time for completion (5 minutes vs. under 30 sec without it)
categories = ['product/productId','review/userId', 'review/profileName','review/helpfulness', 'review/score', 'review/time','review/summary', 'review/text']
i=0
while i < (len(data_foodtext)-1):
    if data_foodtext[i] not in categories  and data_foodtext[i+1] not in categories:
            data_foodtext[i]= data_foodtext[i]+':'+data_foodtext[i+1]
            del data_foodtext[i+1]

    i = i +1



#once all data is correctly split, take data after each keyword
#print(data_foodtext)

productid = []
helpful = []
totalrev = []
score = []
review = []

for index, line in enumerate(data_foodtext):
    if line == 'product/productId':
        string = str(data_foodtext[index+1].replace(' ',''))
        productid.append(string)
    
    if line == 'review/helpfulness':
        revhelp = data_foodtext[index+1].split('/')
        helpful.append(float(str(revhelp[0]).replace(' ','')))
        totalrev.append(float(revhelp[1]))
        
    if line == 'review/score':
        score.append(float(str(data_foodtext[index+1]).replace(' ','')))
        
    if line == 'review/text':
        review.append(data_foodtext[index+1].lstrip())
        
       


CPU times: user 4min 58s, sys: 2.32 s, total: 5min
Wall time: 5min 2s


In [2]:
%%time
food_df = pd.DataFrame({'Product ID': productid, '# People Helpful':helpful, 'Total Reviews': totalrev, 'Score': score, 'Review': review })

print(food_df['Review'][5])

I got a wild hair for taffy and ordered this five pound bag. The taffy was all very enjoyable with many flavors: watermelon, root beer, melon, peppermint, grape, etc. My only complaint is there was a bit too much red/black licorice-flavored pieces (just not my particular favorites). Between me, my kids, and my husband, this lasted only two weeks! I would recommend this brand of taffy -- it was a delightful treat.
CPU times: user 378 ms, sys: 48.5 ms, total: 426 ms
Wall time: 416 ms


## Problem 1(b).  Analyzing review text.

🎯 Add columns to your DataFrame for 
* the length of a review, 
* the number of exclamation points in a review, and 
* the fraction of people who rated a review helpful. 

###### Notes 

You should calculate the fraction who rated a review helpful using two of the columns you made in part 1a.  
* You must convert from text fractions to floating point numbers.  For example, a ratio of 1 helpful rating out of 5 total ratings should be entered as 0.2, not the string ```1/5```.  This floating point number must not be rounded.
* If no people voted on whether a problem was helpful, the corresponding entry in your percentage helpful column should be ```NaN```.  0/0 should generally turn into a `nan` when computing.

In [3]:
%%time
food_df['Review Length'] = food_df['Review'].apply(lambda x: len(x))
food_df['# Exclamation'] = food_df['Review'].apply(lambda x: x.count('!'))
food_df['Helpful Fraction'] = food_df['# People Helpful'] / food_df['Total Reviews']

print(food_df.head(20))




    Product ID  # People Helpful  Total Reviews  Score  \
0   B001E4KFG0               1.0            1.0    5.0   
1   B00813GRG4               0.0            0.0    1.0   
2   B000LQOCH0               1.0            1.0    4.0   
3   B000UA0QIQ               3.0            3.0    2.0   
4   B006K2ZZ7K               0.0            0.0    5.0   
5   B006K2ZZ7K               0.0            0.0    4.0   
6   B006K2ZZ7K               0.0            0.0    5.0   
7   B006K2ZZ7K               0.0            0.0    5.0   
8   B000E7L2R4               1.0            1.0    5.0   
9   B00171APVA               0.0            0.0    5.0   
10  B0001PB9FE               1.0            1.0    5.0   
11  B0009XLVG0               4.0            4.0    5.0   
12  B0009XLVG0               1.0            1.0    1.0   
13  B001GVISJM               2.0            2.0    4.0   
14  B001GVISJM               4.0            5.0    5.0   
15  B001GVISJM               4.0            5.0    5.0   
16  B001GVISJM

## Problem 1(c).  Summary statistics.

🎯 Compute these using Python:

1. How many reviews are in the data set?  
2. What is the average length of a review (in characters)?  
3. What is the average rating?  
4. What is the greatest number of exclamation marks used in a single review?  

Use the pandas package to answer these questions.

In [4]:
%%time
total_reviews = food_df.count(0)
print(total_reviews)

ave_len_review = food_df['Review Length'].mean()
print(ave_len_review)

ave_rating = food_df['Score'].mean()
print(ave_rating)

max_exc = food_df['# Exclamation'].max()
print(max_exc)

index_max_exc = food_df['# Exclamation'].idxmax()
print(index_max_exc)

print(food_df['Review'][128758])
print(food_df['Review'][128758].count('!'))

Product ID          568454
# People Helpful    568454
Total Reviews       568454
Score               568454
Review              568454
Review Length       568454
# Exclamation       568454
Helpful Fraction    298402
dtype: int64
419.2579135690839
4.183198640523243
84
128758
So there I was facing a robot horde low on energy and barely able to remain standing!!!! Then out of nowhere came the delivery man with my package from amazon!!!!!!!!!! That was it!?!?!?!!?! I opened the package and drank the delicious E-tank and I could feel my power go over 9,000! It was then the ZX was tagged on my name and I crushed the dark forces!!!!!!!!!!!!!!!!!!!!!!!!!!!! Thank you AMAZON AND CAPCOM!!!!!!!!!!!!!!!!!!!!!!!!!!!!! With this amazing product we all can be heroes!!!!!! Or have a tasty drink
84
CPU times: user 235 ms, sys: 45.7 ms, total: 280 ms
Wall time: 203 ms


1. Number of reviews: 568,454
2. Average length of review: 419 characters
3. Average rating: 4.18
4. Max exclamation points in single review: 84

## Problem 1(d).  Export.

🎯 Save your DataFrame as a `.csv` file suitable for future analysis in R.  

###### Requirements

* Your .csv file must not include the review text column, as the presence of commas and quotation marks will make reading the file difficult.  
* You should also convert entries from ```NaN``` to the empty string before saving.

In [5]:
%%time
import numpy as np
food_df['Helpful Fraction'] = food_df['Helpful Fraction'].replace(np.nan, '')
#food_df.head(20)

pd.concat([food_df['Product ID'], food_df['# People Helpful'], food_df['Total Reviews'], food_df['Score'], food_df['Review Length'],food_df['# Exclamation'],food_df['Helpful Fraction']], axis =1).to_csv('food_output.csv', index =False)

CPU times: user 3.42 s, sys: 127 ms, total: 3.55 s
Wall time: 3.51 s
